In [2]:
!pip install kaggle
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!echo '{"username":"midlow","key":"19e4a7b3c26e4d040a5179c6b36318cd"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d dhruvildave/wikibooks-dataset
!unzip wikibooks-dataset.zip
!rm wikibooks-dataset.zip
!rm *.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
rm: cannot remove '/root/.kaggle': No such file or directory
 99% 1.81G/1.82G [00:15<00:00, 234MB/s]
100% 1.82G/1.82G [00:15<00:00, 122MB/s]
Archive:  wikibooks-dataset.zip
  inflating: wikibooks.sqlite        
rm: cannot remove '*.zip': No such file or directory


In [3]:
!pip install keras-nlp

In [4]:
import sqlite3
import keras_nlp
import nltk
from nltk.tokenize import word_tokenize
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')

con = sqlite3.connect("wikibooks.sqlite")
cur = con.cursor()
res = cur.execute("SELECT body_text FROM en")
print(res.fetchone())
data = res.fetchall()
text = ""
for d in data[:2000]:
    text += d[0]
corpus = [t[0] for t in data]
del data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


('Front Page: Radiation Oncology | RTOG Trials | Randomized Trials\n\n\n\n\nNon-Hodgkin lymphoma: Main Page  | Randomized\nOverview: Overview  | \nFollicular |\nDiffuse large B-cell |\nMALT |\nNodal marginal zone |\nMantle cell |\nCLL/SLL |\nLymphoblastic |\nBurkitt |\nNK/T cell |\nAnaplastic large cell |\nPrimary CNS Lymphoma\nTreatment:\nAggressive |\nSpecific sites |\nRadioimmunotherapy\n\n\n\nChronic Lymphocytic Leukemia and Small Lymphocytic Lymphoma (CLL/SLL)\n\n\nContents\n\n1 Overview\n2 Staging\n3 Classification\n4 Richter\'s transformation\n5 Radiation Therapy\n6 Reviews\n\n\n\nOverview[edit\xa0| edit source]\nCLL is the most common leukemia among adults in Western world\nIt is characterized by accumulation of mature B-cells\nCLL molecular phenotype: CD5+, CD23+, surface Ig weak, CD79b weak/absent, FMC7 neg.\nDiagnosis: lymphocytosis (often >5 x 10^9 / L, but not an absolute cutoff)\nRisk factors are male sex, advanced age, white race, and family history of CLL or lymphoproli

##Посимвольная токенизация

In [5]:
text = text[:2000]  # берём 2000 символов
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', char_level=True )
tokenizer.fit_on_texts(text)

sequences = tokenizer.texts_to_sequences(text)
input_data = np.array(sequences[:-1])
target_data = np.array(sequences[1:])

In [7]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Dropout

In [8]:
char_model1 = tf.keras.models.Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=10, input_length=input_data.shape[1]),
    LSTM(units=32, return_sequences=True),
    TimeDistributed(Dense(units=len(tokenizer.word_index)+1, activation='softmax'))
])

char_model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
char_model1.fit(input_data, target_data, epochs=10, verbose=0)

In [12]:
generated_text = 'l'
for i in range(5):
    sequence = tokenizer.texts_to_sequences(generated_text)[-input_data.shape[1]:]
    sequence = np.pad(sequence, (0, input_data.shape[1]-len(sequence)), 'constant', constant_values=0)
    prediction = char_model1.predict(np.array([sequence]))
    next_token = np.argmax(prediction[0][-1])
    generated_text += tokenizer.index_word[next_token]

print(generated_text)

1/1 [==============================] - 0s 57ms/step
l e e 


In [13]:
char_model2 = tf.keras.models.Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=10, input_length=input_data.shape[1]),
    LSTM(units=128, return_sequences=True),
    LSTM(units=64, return_sequences=True),
    LSTM(units=32, return_sequences=True),
    TimeDistributed(Dense(units=len(tokenizer.word_index)+1, activation='softmax'))
])

char_model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
char_model2.fit(input_data, target_data, epochs=10, verbose=0)

In [15]:
generated_text = 'l'
for i in range(5):
    sequence = tokenizer.texts_to_sequences(generated_text)[-input_data.shape[1]:]
    sequence = np.pad(sequence, (0, input_data.shape[1]-len(sequence)), 'constant', constant_values=0)
    prediction = char_model2.predict(np.array([sequence]))
    next_token = np.argmax(prediction[0][-1])
    generated_text += tokenizer.index_word[next_token]

print(generated_text)

1/1 [==============================] - 0s 25ms/step
l t t 


##Пословная токенизация

In [16]:
import re

raw_corpus = corpus[:5]  # берём 5 текстов
corpus = []
for t in raw_corpus:
  corpus.append(re.sub(r'[^a-zA-Z0-9 ]', ' ', t))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]

In [17]:
word_model = tf.keras.Sequential()
word_model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
word_model.add(LSTM(256))
word_model.add(Dense(total_words, activation='softmax'))

word_model.compile(loss='categorical_crossentropy', optimizer='adam')
word_model.fit(predictors, tf.keras.utils.to_categorical(label, num_classes=total_words), epochs=10)

Epoch 1/10
35/35 [==============================] - 14s 330ms/step - loss: 5.8292
Epoch 2/10
35/35 [==============================] - 5s 143ms/step - loss: 5.5048
Epoch 3/10
35/35 [==============================] - 7s 187ms/step - loss: 5.4322
Epoch 4/10
35/35 [==============================] - 3s 97ms/step - loss: 5.3841
Epoch 5/10
35/35 [==============================] - 2s 71ms/step - loss: 5.3529
Epoch 6/10
35/35 [==============================] - 4s 106ms/step - loss: 5.3692
Epoch 7/10
35/35 [==============================] - 2s 67ms/step - loss: 5.7332
Epoch 8/10
35/35 [==============================] - 2s 71ms/step - loss: 5.1238
Epoch 9/10
35/35 [==============================] - 2s 60ms/step - loss: 5.0579
Epoch 10/10
35/35 [==============================] - 2s 43ms/step - loss: 4.9603


In [18]:
seed_text = "I want to"
next_words = 1

for _ in range(next_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen=max_sequence_len-1, padding='pre')
    prob_distribution = word_model.predict(encoded)[0]
    prediction = np.argmax(prob_distribution)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == prediction:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 408ms/step
I want to edit


In [27]:
word_model2 = tf.keras.Sequential()
word_model2.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
word_model2.add(LSTM(128, return_sequences=True, activation='tanh'))
word_model2.add(LSTM(64, return_sequences=True, activation='tanh'))
word_model2.add(LSTM(32))
word_model2.add(Dense(total_words, activation='softmax'))

word_model2.compile(loss='categorical_crossentropy', optimizer='adam')
word_model2.fit(predictors, tf.keras.utils.to_categorical(label, num_classes=total_words), epochs=10)

Epoch 1/10
35/35 [==============================] - 16s 286ms/step - loss: 6.0048
Epoch 2/10
35/35 [==============================] - 5s 144ms/step - loss: 5.6163
Epoch 3/10
35/35 [==============================] - 6s 162ms/step - loss: 5.4363
Epoch 4/10
35/35 [==============================] - 4s 121ms/step - loss: 5.4028
Epoch 5/10
35/35 [==============================] - 3s 82ms/step - loss: 5.3928
Epoch 6/10
35/35 [==============================] - 4s 130ms/step - loss: 5.3891
Epoch 7/10
35/35 [==============================] - 3s 75ms/step - loss: 5.3862
Epoch 8/10
35/35 [==============================] - 2s 68ms/step - loss: 5.3854
Epoch 9/10
35/35 [==============================] - 2s 67ms/step - loss: 5.3832
Epoch 10/10
35/35 [==============================] - 2s 68ms/step - loss: 5.3839


In [32]:
seed_text = "I want to"
next_words = 1

for _ in range(next_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen=max_sequence_len-1, padding='pre')
    prob_distribution = word_model2.predict(encoded)[0]
    prediction = np.argmax(prob_distribution)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == prediction:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 66ms/step
I want to edit


##Двунаправеленная LSTM

In [33]:
from tensorflow.keras.layers import Bidirectional
word_model3 = tf.keras.Sequential()
word_model3.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
word_model3.add(Bidirectional(LSTM(128, return_sequences=True, activation='tanh')))
word_model3.add(Bidirectional(LSTM(64, return_sequences=True, activation='tanh')))
word_model3.add(Bidirectional(LSTM(32)))
word_model3.add(Dense(total_words, activation='softmax'))

word_model3.compile(loss='categorical_crossentropy', optimizer='adam')
word_model3.fit(predictors, tf.keras.utils.to_categorical(label, num_classes=total_words), epochs=10)

Epoch 1/10
35/35 [==============================] - 21s 227ms/step - loss: 5.9079
Epoch 2/10
35/35 [==============================] - 8s 231ms/step - loss: 5.5013
Epoch 3/10
35/35 [==============================] - 7s 186ms/step - loss: 5.4238
Epoch 4/10
35/35 [==============================] - 8s 244ms/step - loss: 5.4079
Epoch 5/10
35/35 [==============================] - 5s 142ms/step - loss: 5.4036
Epoch 6/10
35/35 [==============================] - 5s 152ms/step - loss: 5.4005
Epoch 7/10
35/35 [==============================] - 4s 120ms/step - loss: 5.3947
Epoch 8/10
35/35 [==============================] - 5s 131ms/step - loss: 5.3924
Epoch 9/10
35/35 [==============================] - 5s 141ms/step - loss: 5.3935
Epoch 10/10
35/35 [==============================] - 4s 117ms/step - loss: 5.3915


In [35]:
seed_text = "I want to"
next_words = 1

for _ in range(next_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen=max_sequence_len-1, padding='pre')
    prob_distribution = word_model3.predict(encoded)[0]
    prediction = np.argmax(prob_distribution)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == prediction:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 102ms/step
I want to edit
